In [4]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [5]:
# Data Augmentation Parameters
batchsize = 64  # Increase batch size for faster training
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, shear_range=0.2,
                                   zoom_range=0.2, width_shift_range=0.2,
                                   height_shift_range=0.2, validation_split=0.2)

In [6]:
# Data Generators
train_data = train_datagen.flow_from_directory(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\data\train',
                                               target_size=(224, 224), batch_size=batchsize,
                                               class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\data\train',
                                                    target_size=(224, 224), batch_size=batchsize,
                                                    class_mode='categorical', subset='validation')


Found 54336 images belonging to 2 classes.
Found 13583 images belonging to 2 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\data\test',
                                             target_size=(224, 224), batch_size=batchsize,
                                             class_mode='categorical')

Found 16979 images belonging to 2 classes.


In [5]:
# Base model
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(224, 224, 3)))

# Model head
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(128, activation='relu')(hmodel)  # Increase number of neurons
hmodel = Dropout(0.5)(hmodel)  # Increase dropout rate to reduce overfitting
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

# Combined model
model = Model(inputs=bmodel.input, outputs=hmodel)

# Freeze layers in base model
for layer in bmodel.layers:
    layer.trainable = False

In [6]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 28,364,898 (108.20 MB)

 Trainable params: 6,562,114 (25.03 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [13]:
# Callbacks
checkpoint_path = r'C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels.keras'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)

callbacks = [checkpoint, earlystop, learning_rate]

In [14]:
# Adjust Learning Rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  # Reduce learning rate for smoother convergence
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning (unfreeze some layers for fine-tuning)
for layer in bmodel.layers[-30:]:
    layer.trainable = True

In [15]:
# Train the model with increased number of epochs
model.fit(train_data,
          steps_per_epoch=train_data.samples // batchsize,
          validation_data=validation_data,
          validation_steps=validation_data.samples // batchsize,
          callbacks=callbacks,
          epochs=30) 

Epoch 1/30
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9657 - loss: 0.1053
Epoch 1: val_loss improved from inf to 0.23782, saving model to C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels.keras
849/849 ━━━━━━━━━━━━━━━━━━━━ 2100s 2s/step - accuracy: 0.9657 - loss: 0.1053 - val_accuracy: 0.9426 - val_loss: 0.2378 - learning_rate: 1.0000e-04
Epoch 2/30


C:\Users\rajat\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss did not improve from 0.23782
849/849 ━━━━━━━━━━━━━━━━━━━━ 40s 47ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8000 - val_loss: 0.7249 - learning_rate: 1.0000e-04
Epoch 3/30
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9704 - loss: 0.0915
Epoch 3: val_loss improved from 0.23782 to 0.22525, saving model to C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels.keras
849/849 ━━━━━━━━━━━━━━━━━━━━ 1964s 2s/step - accuracy: 0.9704 - loss: 0.0915 - val_accuracy: 0.9433 - val_loss: 0.2253 - learning_rate: 1.0000e-04
Epoch 4/30

Epoch 4: val_loss improved from 0.22525 to 0.02819, saving model to C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels.keras
849/849 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0282 - learning_rate: 1.0000e-04
Epoch 5/30
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9758 - loss: 0.0767
Epoch 5: val_loss did not improve from 0.

In [11]:
import numpy as np
from tensorflow.keras.models import load_model

model = load_model(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels.keras')

# Evaluate the model on the training data generator
loss_tr,acc_tr = model.evaluate(train_data)

# Print the training accuracy and loss
print("Training Accuracy:", acc_tr)
print("Training Loss:", loss_tr)


849/849 ━━━━━━━━━━━━━━━━━━━━ 846s 991ms/step - accuracy: 0.9829 - loss: 0.0483
Training Accuracy: 0.9829394817352295
Training Loss: 0.04845675081014633


In [14]:
# Evaluate the model on the validation data generator
loss_vr,acc_vr = model.evaluate(validation_data)

# Print the validation accuracy and loss
print("Validation Accuracy:", acc_vr)
print("Validation Loss:", loss_vr)

# Evaluate the model on the test data generator
loss_test, acc_test = model.evaluate(test_data)

# Print the test accuracy and loss
print("Test Accuracy:", acc_test)
print("Test Loss:", loss_test)

213/213 ━━━━━━━━━━━━━━━━━━━━ 212s 994ms/step - accuracy: 0.9472 - loss: 0.2017
Validation Accuracy: 0.9451520442962646
Validation Loss: 0.2135116308927536
266/266 ━━━━━━━━━━━━━━━━━━━━ 294s 1s/step - accuracy: 0.9778 - loss: 0.0693
Test Accuracy: 0.9772071242332458
Test Loss: 0.07431238889694214


In [34]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc
checkpoint_path = r'C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels.keras'
# Load the best model
model.load_weights(checkpoint_path)

# Extract metrics from the model checkpoint
train_accuracy = model.history.history['accuracy']
val_accuracy = model.history.history['val_accuracy']
train_loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

# Plot training and validation accuracy/loss curves
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(train_accuracy)
plt.plot(val_accuracy)
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(train_loss)
plt.plot(val_loss)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.show()


# Plot training and validation accuracy/loss curves
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.show()

# Generate confusion matrix for test data
confusion_mat = confusion_matrix(true_classes, predicted_classes)

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(confusion_mat, cmap='Blues', interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(true_classes, predictions[:, 1])
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()


KeyError: 'accuracy'

In [ ]:
print("Test Accuracy:", acc_pred)
print("Test Loss:", loss_pred)

In [20]:
predicted_classes

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [21]:
true_classes

array([0, 0, 0, ..., 1, 1, 1])

In [22]:
# Generate confusion matrix for test data
confusion_mat = confusion_matrix(true_classes, predicted_classes)


In [23]:
confusion_mat

array([[4212, 4177],
       [4218, 4372]], dtype=int64)

In [24]:
# Map the predicted classes to the corresponding class labels
predicted_class_labels = [class_labels[i] for i in predicted_classes]

# Get true labels for the test data
true_classes = test_data.classes

In [25]:
predicted_class_labels

['closed',
 'open',
 'closed',
 'open',
 'closed',
 'open',
 'closed',
 'open',
 'closed',
 'closed',
 'open',
 'open',
 'closed',
 'closed',
 'open',
 'open',
 'open',
 'open',
 'closed',
 'closed',
 'closed',
 'closed',
 'open',
 'open',
 'open',
 'open',
 'closed',
 'closed',
 'open',
 'closed',
 'closed',
 'closed',
 'open',
 'closed',
 'closed',
 'open',
 'open',
 'open',
 'closed',
 'closed',
 'closed',
 'open',
 'closed',
 'closed',
 'open',
 'closed',
 'open',
 'open',
 'open',
 'closed',
 'open',
 'open',
 'open',
 'closed',
 'closed',
 'closed',
 'open',
 'open',
 'open',
 'closed',
 'open',
 'closed',
 'open',
 'open',
 'closed',
 'closed',
 'open',
 'open',
 'open',
 'open',
 'closed',
 'open',
 'open',
 'open',
 'open',
 'open',
 'open',
 'open',
 'open',
 'closed',
 'closed',
 'closed',
 'closed',
 'open',
 'open',
 'closed',
 'open',
 'open',
 'open',
 'closed',
 'closed',
 'open',
 'open',
 'open',
 'closed',
 'closed',
 'closed',
 'closed',
 'open',
 'open',
 'open',
 

In [26]:
true_classes

array([0, 0, 0, ..., 1, 1, 1])

In [28]:
# Convert predicted class labels from string to numerical labels
predicted_class_labels_numeric = [class_labels.index(label) for label in predicted_class_labels]

# Now, use the numerical labels for generating the confusion matrix and classification report


In [29]:
# Generate confusion matrix for test data
confusion_mat = confusion_matrix(true_classes, predicted_class_labels_numeric)

# Generate classification report for test data
class_report = classification_report(true_classes, predicted_class_labels_numeric)

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_mat)

# Print classification report
print("\nClassification Report:")
print(class_report)


Confusion Matrix:
[[4212 4177]
 [4218 4372]]

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.50      0.50      8389
           1       0.51      0.51      0.51      8590

    accuracy                           0.51     16979
   macro avg       0.51      0.51      0.51     16979
weighted avg       0.51      0.51      0.51     16979

